In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col,max,min,count,sum,avg,stddev_pop,hour,countDistinct,expr,stddev,window,column
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row, SparkSession, SQLContext
import math
from operator import add
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.master("local[*]").appName('Test').getOrCreate()
sc.setSystemProperty('spark.executor.memory', '6g')
sc.setSystemProperty('spark.driver.memory', '6g')

22/11/13 22:24:08 WARN Utils: Your hostname, Spark resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/11/13 22:24:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/13 22:24:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/13 22:24:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/13 22:24:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/13 22:24:12 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/13 22:24:12 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/11/13 22:24:12 WARN 

In [2]:
from pyspark.sql.functions import col, floor, sequence, lit
#Using a small demo dataset with a small subset of data. Contains about 30 data points
df = spark.read.csv("/home/vboxuser/Downloads/datasetPP4.csv")
df = df.select(col('_c0').alias('x'), col('_c1').alias('y'))
df = df.select('x', 'y')
df.show(5)

+----+----+
|   x|   y|
+----+----+
|1679| 467|
|7687|8377|
|7257|   0|
|4055|9578|
|5830|8140|
+----+----+
only showing top 5 rows



In [3]:
#row -- ((cell / 500) + 1), col -- (cell % 500)
#north = ID - 500, NE = ID - 499, E = ID+1, SE = ID+501, S = ID + 500, SW = ID + 499, NW = ID-501
#Check that these values dont go below 0 --> check edge case
#R*500 = max cell num for that row
#C*500 = max cell num for that col
#relative density = X.count/avg(Yn.count) -- y is the neighbors of x

#Step 1: Associate points with respective cells
rc = df
for field in rc.columns:
    #print(field)
    rc = rc.withColumn(field, floor(col(field)/20))
rc.show(2)

+---+---+
|  x|  y|
+---+---+
| 83| 23|
|384|418|
+---+---+
only showing top 2 rows



In [4]:
#row -- ((cell / 500) + 1), col -- (cell % 500)
#Step 2
from pyspark.sql.functions import concat, col, lit
#Replace 31 with df_len before turning this in! COME BACK
df_len = 250001
freq = 1
ref = spark.range(1, 1000000).toDF("Cell_ID")
ref = ref.withColumn("r", floor((ref.Cell_ID/500) + 1))
ref = ref.withColumn("c", floor(ref.Cell_ID % 500))
ref = ref.withColumn("rc",concat(col("r"), lit(","), col("c")))
ref.show(3)

+-------+---+---+---+
|Cell_ID|  r|  c| rc|
+-------+---+---+---+
|      1|  1|  1|1,1|
|      2|  1|  2|1,2|
|      3|  1|  3|1,3|
+-------+---+---+---+
only showing top 3 rows



In [5]:
#R*500 = max cell num for that row
#C*500 = max cell num for that col
#Step 3
ref = ref.withColumn("Max_Row", ref.r * 500)
ref = ref.withColumn("Max_Cell", ref.c * 500)
ref.show(3)

+-------+---+---+---+-------+--------+
|Cell_ID|  r|  c| rc|Max_Row|Max_Cell|
+-------+---+---+---+-------+--------+
|      1|  1|  1|1,1|    500|     500|
|      2|  1|  2|1,2|    500|    1000|
|      3|  1|  3|1,3|    500|    1500|
+-------+---+---+---+-------+--------+
only showing top 3 rows



In [6]:
#Combine the dataframes into one
from pyspark.sql.functions import monotonically_increasing_id
rc = rc.withColumn("id1", monotonically_increasing_id())
ref = ref.withColumn("id2", monotonically_increasing_id())
ref = ref.join(rc, col("id1") == col("id2"), "inner").drop("id1", "id2")
ref.show(100)

+-------+---+---+-----+-------+--------+---+---+
|Cell_ID|  r|  c|   rc|Max_Row|Max_Cell|  x|  y|
+-------+---+---+-----+-------+--------+---+---+
|      1|  1|  1|  1,1|    500|     500| 83| 23|
|      2|  1|  2|  1,2|    500|    1000|384|418|
|      3|  1|  3|  1,3|    500|    1500|362|  0|
|      4|  1|  4|  1,4|    500|    2000|202|478|
|      5|  1|  5|  1,5|    500|    2500|291|407|
|      6|  1|  6|  1,6|    500|    3000| 53|134|
|      7|  1|  7|  1,7|    500|    3500|262|352|
|      8|  1|  8|  1,8|    500|    4000| 83|302|
|      9|  1|  9|  1,9|    500|    4500|138| 82|
|     10|  1| 10| 1,10|    500|    5000|159|389|
|     11|  1| 11| 1,11|    500|    5500|486|192|
|     12|  1| 12| 1,12|    500|    6000|274|186|
|     13|  1| 13| 1,13|    500|    6500|273| 25|
|     14|  1| 14| 1,14|    500|    7000|253|175|
|     15|  1| 15| 1,15|    500|    7500|192|126|
|     16|  1| 16| 1,16|    500|    8000| 34| 66|
|     17|  1| 17| 1,17|    500|    8500|247|372|
|     18|  1| 18| 1,

In [7]:
#relative density = X.count/avg(Yn.count) -- y is the neighbors of x
#Step 4
from pyspark.sql.functions import count
ref = ref.withColumn("count_xy",concat(col("x"), lit(","), col("y")))
xCount = ref.groupby("count_xy").count()
xCount.show(3)
new = ref.select("Cell_ID", "rc")
new.show(3)
joined = xCount.join(new, col("rc") == col("count_xy"), "inner").drop("count_xy")
joined = joined.select("Cell_ID", "rc", "count")
joined.show(3)
#.select("Cell_ID", "rc", count("xy"))
#xCount.show(3)
#yavg = ref.select(count('y'))
#yavg.show(3)
#yavg = yavg.agg({'y': 'avg'})
#yavg.show(2)
#rc = xCount.withColumn("id1", monotonically_increasing_id())
#rel = yavg.withColumn("id2", monotonically_increasing_id())
#rel = rel.join(rc, col("id1") == col("id2"), "inner").drop("id1", "id2")
#ref = ref.withColumn("Relative_Density", (xCount / yavg))
#ref.show(3)

+--------+-----+
|count_xy|count|
+--------+-----+
| 440,333|    6|
|  55,220|    4|
| 341,449|    2|
+--------+-----+
only showing top 3 rows



+-------+---+
|Cell_ID| rc|
+-------+---+
|      1|1,1|
|      2|1,2|
|      3|1,3|
+-------+---+
only showing top 3 rows



+-------+-----+-----+
|Cell_ID|   rc|count|
+-------+-----+-----+
|      1|  1,1|    2|
|     10| 1,10|    3|
|    100|1,100|    5|
+-------+-----+-----+
only showing top 3 rows



In [8]:
#check = joined.select("rc")
#check.
joined = joined.na.fill(value = 0)

In [9]:
#Now calculate RDI
#north = ID - 500, NE = ID - 499, E = ID+1, SE = ID+501, S = ID + 500, SW = ID + 499, NW = ID-501
def neighbor_search(x):
    rowcol = x.split(",")
    row = rowcol[0]
    column = rowcol[1]
    #rc = x.rc
    #rcs = x.split(",")
    #row = rcs[0]
    #column = rcs[1]
    
    totalYSum = 0
    totalYCount = 0
    # N
    lookx = row-1
    looky = column
    if (lookx - 1 > 0):
        # grab r,c from table and get count
        try:
            totalYSum += joined.select("count").where(col("rc") == str(lookx) + "," + str(looky)).collect()[0]['count']
        except:
            print("Skip")
        totalYCount += 1
    # NE
    lookx = row-1
    looky = column - 1
    if(lookx - 1 > 0 and looky - 1 > 0):
        ## grab r,c from table and get count
        try:
            totalYSum += joined.select("count").where(col("rc") == str(lookx) + "," + str(looky)).collect()[0]['count']
        except:
            print("Skip")
        totalYCount += 1
    #E
    lookx = row
    looky = row+1
    if(looky <= 500):
        ## grab r,c from table and get count
        try:
            totalYSum += joined.select("count").where(col("rc") == str(lookx) + "," + str(looky)).collect()[0]['count']
        except:
            print("Skip")
        totalYCount += 1
    #SE
    lookx = row + 1
    looky = column + 1
    if(lookx <= 500 and looky <= 500):
        #### grab r,c from table and get coun
        try:
            totalYSum += joined.select("count").where(col("rc") == str(lookx) + "," + str(looky)).collect()[0]['count']
        except:
            print("Skip")
        totalYCount += 1
        
    #S
    lookx = row + 1
    looky = column
    if(lookx <= 500):
        #### grab r,c from table and get coun
        try:
            totalYSum += joined.select("count").where(col("rc") == str(lookx) + "," + str(looky)).collect()[0]['count']
        except:
            print("Skip")
        totalYCount += 1
    
    
    #SW
    lookx = row + 1
    looky = column - 1
    if(lookx <= 500 and looky > 0):
        #### grab r,c from table and get coun
        try:
            totalYSum += joined.select("count").where(col("rc") == str(lookx) + "," + str(looky)).collect()[0]['count']
        except:
            print("Skip")
        totalYCount += 1
    #W
    lookx = row
    looky = column - 1
    if(looky > 0):
        #### grab r,c from table and get coun
        try:
            totalYSum += joined.select("count").where(col("rc") == str(lookx) + "," + str(looky)).collect()[0]['count']
        except:
            print("Skip")
        totalYCount += 1
    
    #NW
    lookx = row - 1
    looky = column - 1
    if(looky > 0 and lookx > 0):
        try:
            totalYSum += joined.select("count").where(col("rc") == str(lookx) + "," + str(looky)).collect()[0]['count']
        except:
            print("Skip")
        totalYCount += 1
        
    try:
        xcount = joined.select("count").where(col("rc") == str(row) + "," + str(column)).collect()[0]['count']
    except:
        return str(0)
        
    return str(xcount / (totalYSum / totalYCount))

print(joined)

test_list = []
test = joined.select("rc").rdd.toLocalIterator()
for row in test:
    test_list.append(str(row[0]))
                     
parList = sc.parallelize(test_list)
rdis = parList.mapPartitions(lambda x: neighbor_search(x))
print(rdis)
#rdisdf = rdis.toDF()
#rdisfd.show(5)



#rdd2 = joined.rdd.map(lambda x: neighbor_search(x))
#rdd2 = rdd2.toDF()
#rdd2.show(5)

#joined.foreach(lambda x: neighbor_search(x))
#joined = joined.withColumn("rdi", rdi)
# for each row in joined
# get value of neighbor(search) given the row r,c
# Get the count for that r,c
# Divide count (X.Count) / YAverage)
#map that to r,c -> RDI


SyntaxError: unexpected EOF while parsing (4111194409.py, line 129)

In [ ]:
lookx = 5
looky = 3
#value = joined.select("count").where(col("rc") == str(lookx) + "," + str(looky)).collect()[0]['count']
value = joined.select("rc").collect()[0]['rc']
print(value)                           